#处理轨迹数据
##读取轨迹数据

In [ ]:
import pickle
import sys
D=1600
W=12
ID=4
def read_data(D,W):
    '''
    通过D,和W的大小来区分和读取数据，并返回四个数据结果。
    '''
    name=sys.path[0]+'\data\\'+'D='+str(D)+'_W='+str(W)
    
    with open(name+'_mouse_check.bin','rb') as file:
        check_datas=pickle.load(file)
    with open(name+'_mouse_move.bin','rb') as file:
        move_datas=pickle.load(file)
    with open(name+'_track.bin','rb') as file:
        track_datas=pickle.load(file)
    with open(name+'_information.bin','rb') as file:
        inf=pickle.load(file)
    
    return move_datas,check_datas,track_datas,inf

mouse_move,mouse_check,track,inf=read_data(D,W)
"""
dict_keys(['screen_width', 'screen_height', 'D', 'W', 'cir1', 'cir2'])
"""

In [ ]:
for i in range(len(track)):
    for j in range(len(track[i])):
        print(track[i][j])

In [ ]:
import numpy as np

def get_v_S_T(track_for_ones):
    '''
    计算速度和时间
    '''
    t=0
    T=[]
    for i in range(len(track_for_ones)-1):
        t+=track_for_ones[i+1][0]-track_for_ones[i][0]
        T.append(t)
    s=0
    S=[]#点在轨迹上的位置
    for i in range(len(track_for_ones)-1):
        s+=np.sqrt((track_for_ones[i][1][0]-track_for_ones[i+1][1][0])**2+(track_for_ones[i][1][1]-track_for_ones[i+1][1][1])**2)
        S.append(s)
    v=[]
    a=[]
    for i in range(len(track_for_ones)-1):
        v.append(np.sqrt((track_for_ones[i][1][0]-track_for_ones[i+1][1][0])**2+(track_for_ones[i][1][1]-track_for_ones[i+1][1][1])**2)/(track_for_ones[i+1][0]-track_for_ones[i][0]))
        a.append(np.arctan((track_for_ones[i][1][1]-track_for_ones[i+1][1][1])/(track_for_ones[i][1][0]-track_for_ones[i+1][1][0]+1e-10)))
    return v,S,T



    

最好沿着轨迹绘制图像
因为在轨迹上能够更好地展现速度的变化

In [ ]:
import matplotlib.pyplot as plt
for i in range(len(track)):
    #print(track[i])
    v,S,T=get_v_S_T(track[i])
    
    # 定义滑动窗口大小和滤波后的数据容器
    window_size = 3
    smoothed_data = np.zeros_like(v)

    # 应用移动平均滤波
    for i in range(window_size, len(v)-window_size):
        smoothed_data[i] = np.mean(v[i-window_size:i+window_size+1])
    # 绘制图像
    plt.plot(S, smoothed_data, color='red', linewidth=2.0)
    plt.plot(S, v, color='blue', linewidth=1.0,linestyle='--')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Sine Function')
    plt.grid(True)
    plt.show()
    

In [ ]:
import matplotlib.pyplot as plt

colors=['red','blue','green','yellow','pink','purple','orange','gray','brown',
        'red','blue','green','yellow','pink','purple','orange','gray','brown',
        'red','blue','green','yellow','pink','purple','orange','gray','brown',]

t_a=0
for i in range(len(track)):
    #print(track[i])
    v,S,T=get_v_S_T(track[i])
    
    # 定义滑动窗口大小和滤波后的数据容器
    window_size = 3
    smoothed_data = np.zeros_like(v)

    # 应用移动平均滤波
    
    for j in range(window_size, len(v)-window_size):
        smoothed_data[j] = np.mean(v[j-window_size:j+window_size+1])
    # 绘制图像
    plt.plot(T, S, color=colors[i],  linewidth=2.0,linestyle='--')
    plt.axvline(x=T[-1], color=colors[i], linestyle='--')
    plt.xlabel('T')
    plt.ylabel('S')
    plt.title('Sine Function')
    plt.grid(True)
    t_a+=T[-1]
plt.axvline(x=(t_a)/len(track), color='black', linewidth=2.0)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 生成示例的含噪声的时序数据
t = np.linspace(0, 10, 1000)
y = np.sin(t) + 0.4 * np.random.randn(1000)

# 定义滑动窗口大小和滤波后的数据容器
window_size = 10
smoothed_data = np.zeros_like(y)

# 应用移动平均滤波
for i in range(window_size, len(y)-window_size):
    smoothed_data[i] = np.mean(y[i-window_size:i+window_size+1])

# 绘制原始数据和滤波后的数据
plt.plot(t, y, label='Noisy Data')
plt.plot(t, smoothed_data, label='Smoothed Data')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.title('Moving Average Filter')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

colors=['red','blue','green','yellow','pink','purple','orange','gray','brown',
        'red','blue','green','yellow','pink','purple','orange','gray','brown',]

def polynomial_func(x, *coefficients):
    n = int(len(coefficients)/2)
    result = np.zeros_like(x)
    for i in range(n):
        result += coefficients[i] * np.power(x,i)+coefficients[i+n]
    return result

S=[]
T=[]
for i in range(len(track)):
    #print(track[i])
    _,s,t=get_v_S_T(track[i])
    S.extend(s)
    T.extend(t)

# 使用 zip() 函数将两个列表配对并排序
sorted_pairs = sorted(zip(T, S))

# 解压排序后的配对，重新组合成两个列表
T, S = zip(*sorted_pairs)


# 使用最小二乘法进行拟合
initial_guess = np.zeros(8+8)  # 初始猜测参数
popt, pcov = curve_fit(polynomial_func, np.array(T), np.array(S), p0=initial_guess)

#print(popt)
#print(pcov)
# 绘制原始数据和拟合曲线
plt.scatter(T, S, color=colors[0],)
plt.plot(T, polynomial_func(T, *popt), color=colors[1], label='Fitted Curve')
plt.xlabel('T')
plt.ylabel('S')
plt.title('Curve Fitting Example')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

colors=['red','blue','green','yellow','pink','purple','orange','gray','brown',
        'red','blue','green','yellow','pink','purple','orange','gray','brown',]

def polynomial_func(x, *coefficients):
    n = int(len(coefficients)/2)
    result = np.zeros_like(x)
    for i in range(n):
        result += coefficients[i] * np.power(x,i)+coefficients[i+n]
    return result

S=[]
T=[]
for i in range(len(track)):
    #print(track[i])
    _,s,t=get_v_S_T(track[i])
    S.extend(s)
    T.extend(t)

# 使用 zip() 函数将两个列表配对并排序
sorted_pairs = sorted(zip(S, T))

# 解压排序后的配对，重新组合成两个列表
S,T = zip(*sorted_pairs)


# 使用最小二乘法进行拟合
initial_guess = np.zeros(8+8)  # 初始猜测参数
popt, pcov = curve_fit(polynomial_func, np.array(S), np.array(T), p0=initial_guess)

#print(popt)
#print(pcov)
# 绘制原始数据和拟合曲线
plt.scatter(S, T, color=colors[0],)
plt.plot(S, polynomial_func(S, *popt), color=colors[1], label='Fitted Curve')
plt.xlabel('S')
plt.ylabel('T')
plt.title('Curve Fitting Example')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

colors=['red','blue','green','yellow','pink','purple','orange','gray','brown',
        'red','blue','green','yellow','pink','purple','orange','gray','brown',]

def polynomial_func(x, *coefficients):
    n = int(len(coefficients)/2)
    result = np.zeros_like(x)
    for i in range(n):
        result += coefficients[i] * np.power(x,i)+coefficients[i+n]
    return result

for i in range(len(track)):
    #print(track[i])
    _,S,T=get_v_S_T(track[i])


    '''    # 使用 zip() 函数将两个列表配对并排序
    sorted_pairs = sorted(zip(S, T))

    # 解压排序后的配对，重新组合成两个列表
    S,T = zip(*sorted_pairs)'''


    # 使用最小二乘法进行拟合
    initial_guess = np.zeros(8+8)  # 初始猜测参数
    popt, pcov = curve_fit(polynomial_func, np.array(S), np.array(T), p0=initial_guess)

    #print(popt)
    #print(pcov)
    # 绘制原始数据和拟合曲线
    plt.scatter(S, T, color=colors[0],)
    plt.plot(S, polynomial_func(S, *popt), color=colors[1], label='Fitted Curve')
    plt.xlabel('S')
    plt.ylabel('T')
    plt.title('Curve Fitting Example')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 示例轨迹数据
x = np.linspace(0, 2 * np.pi, 100)
y = np.sin(x)

# 计算曲率
dS = np.gradient(x)
dT = np.gradient(y)
d2S = np.gradient(dS)
d2T = np.gradient(dT)
curvature = np.abs(dS * d2T - dT * d2S) / np.power(dS**2 + dT**2, 1.5)

# 设置曲率阈值，判断转折点
threshold = 0.99
turning_points = np.where(curvature > threshold)[0]

# 绘制轨迹和转折点
plt.plot(x, y, label='Trajectory')
plt.scatter(x[turning_points], y[turning_points], color='red', label='Turning Points')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Trajectory with Turning Points')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

colors=['red','blue','green','yellow','pink','purple','orange','gray','brown',
        'red','blue','green','yellow','pink','purple','orange','gray','brown',]

def polynomial_func(x, *coefficients):
    n = int(len(coefficients)/2)
    result = np.zeros_like(x)
    for i in range(n):
        result += coefficients[i] * np.power(x,i)+coefficients[i+n]
    return result

for i in range(len(track)):
    #print(track[i])
    _,S,T=get_v_S_T(track[i])


    # 使用 zip() 函数将两个列表配对并排序
    sorted_pairs = sorted(zip(S, T))

    # 解压排序后的配对，重新组合成两个列表
    S,T = zip(*sorted_pairs)

    # 计算曲率
    dS = np.gradient(S)
    dT = np.gradient(T)
    d2S = np.gradient(dS)
    d2T = np.gradient(dT)
    curvature = np.abs(dS * d2T - dT * d2S) / np.power(dS**2 + dT**2, 1.5)

    # 设置曲率阈值，判断转折点
    threshold = 0.9
    turning_points = np.array(np.where(curvature > threshold)[0])
    
    # 使用最小二乘法进行拟合
    initial_guess = np.zeros(8+8)  # 初始猜测参数
    popt, pcov = curve_fit(polynomial_func, np.array(S), np.array(T), p0=initial_guess)

    # 绘制原始数据和拟合曲线
    
    plt.scatter(S, T, color=colors[0],)
    plt.plot(S, polynomial_func(S, *popt), color=colors[1], label='Fitted Curve')
    plt.scatter(np.array(S)[turning_points], np.array(T)[turning_points], color='green', label='Turning Points')
    plt.xlabel('S')
    plt.ylabel('T')
    plt.title('Curve Fitting Example')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

colors=['red','blue','green','yellow','pink','purple','orange','gray','brown',
        'red','blue','green','yellow','pink','purple','orange','gray','brown',]

def polynomial_func(x, *coefficients):
    n = int(len(coefficients)/2)
    result = np.zeros_like(x)
    for i in range(n):
        result += coefficients[i] * np.power(x,i)+coefficients[i+n]
    return result

for i in range(len(track)):
    #print(track[i])
    _,S,T=get_v_S_T(track[i])


    # 使用 zip() 函数将两个列表配对并排序
    sorted_pairs = sorted(zip(S, T))

    # 解压排序后的配对，重新组合成两个列表
    T,S = zip(*sorted_pairs)

    # 计算曲率
    dS = np.gradient(S)
    dT = np.gradient(T)
    d2S = np.gradient(dS)
    d2T = np.gradient(dT)
    curvature = np.abs(dS * d2T - dT * d2S) / np.power(dS**2 + dT**2, 1.5)

    # 设置曲率阈值，判断转折点
    threshold = 0.1
    turning_points = np.array(np.where(curvature > threshold)[0])
    
    # 使用最小二乘法进行拟合
    initial_guess = np.zeros(8+8)  # 初始猜测参数
    popt, pcov = curve_fit(polynomial_func, np.array(S), np.array(T), p0=initial_guess)

    # 绘制原始数据和拟合曲线
    
    plt.scatter(S, T, color=colors[0],)
    plt.plot(S, polynomial_func(S, *popt), color=colors[1], label='Fitted Curve')
    plt.scatter(np.array(S)[turning_points], np.array(T)[turning_points], color='green', label='Turning Points')
    plt.xlabel('S')
    plt.ylabel('T')
    plt.title('Curve Fitting Example')
    plt.legend()
    plt.grid(True)
    plt.show()